# wav2vec-u CV-sv - prepare text

> "Running prepare_text.sh for wav2vec-u on Common Voice Swedish"

- toc: false
- branch: master
- badges: false
- hidden: true
- categories: [kaggle, wav2vec-u]

Original [here](https://www.kaggle.com/jimregan/wav2vec-u-cv-swedish-text-prep)

In [ ]:
%cd /opt

/opt


In [ ]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [ ]:
%cd /tmp

/tmp


In [ ]:
!git clone https://github.com/pytorch/fairseq/

In [ ]:
%%capture
!pip install phonemizer

In [ ]:
%%capture
!pip install git+https://github.com/pytorch/fairseq/

In [ ]:
%%capture
!apt-get -y install espeak

In [ ]:
!git clone https://github.com/kpu/kenlm

In [ ]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [ ]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [ ]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [ ]:
!cat /kaggle/input/wav2vec-u-cv-swedish-audio/*.wrd | grep -v '^$' | sort| uniq > /kaggle/working/sentences.txt


In [ ]:
%cd fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [ ]:
%%capture
!apt-get -y install zsh

In [ ]:
!mkdir /kaggle/working/preppedtext

In [ ]:
%cd scripts

/tmp/fairseq/examples/wav2vec/unsupervised/scripts


The next part requires a FastText language id model; I don't know where the 187 language model comes from, but there is a model for 176 languages [here](https://fasttext.cc/docs/en/language-identification.html#content)

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

In [ ]:
!cat normalize_and_filter_text.py|sed -e 's/187/176/' > tmp
!mv tmp normalize_and_filter_text.py

In [ ]:
# Needed to see what's going wrong
os.environ['HYDRA_FULL_ERROR'] = '1'

In [ ]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib:/opt/kaldi/tools/openfst-1.6.7/lib:/opt/kaldi/src/lib'

There are two lines with missing variables in `prepare_text.sh` - [pull request](https://github.com/pytorch/fairseq/pull/3569) - so replace the file.

While I'm replacing the file: most of the first part of the script is unneeded, as I already have a phonetic dictionary, so I'm using that instead.

With the calls of the `preprocess.py` script, make sure to check the threshold: there's a divide by zero if the threshold is set too high.

Config options for `kaldi_initializer.py`

- `in_labels`: a naming component, for the Kaldi lexicons/fsts (required)
- `wav2letter_lexicon`: path to wav2letter lexicon
- `out_labels`: a naming component, for the Kaldi lexicons/fsts: set to `in_label` if missing
- `kaldi_root`: path to Kaldi: `/opt/kaldi` for my kaggle image
- `fst_dir`: path where generated fsts will be saved
- `data_dir`: path to phones data
- `lm_arpa`: path to the lm in ARPA format
- `blank_symbol`: CTC blank symbol (`<s>` here)
- `silence_symbol`: Kaldi symbol for silence (`<SIL>` is set for two of the scripts)

A config file needs to exist for this, even though the options set in it seem to be ignored.

In [ ]:
!mkdir /tmp/fairseq/examples/speech_recognition/kaldi/config/

In [ ]:
%%writefile /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml
kaldi_root: "/opt/kaldi"

Writing /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml


In [ ]:
%%writefile prepare_text.sh
#!/usr/bin/env zsh
# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

lg=$1
text_path=$2
target_dir=$3

#ph_lg=${lg:l}
#if test "$lg" = 'fr'; then
#  ph_lg='fr-fr'
#elif test "$lg" = 'en'; then
#  ph_lg='en-us'
#elif test "$lg" = 'pt'; then
#  ph_lg='pt-br'
#fi
ph_lg="sv"

echo $lg
echo $ph_lg
echo $text_path
echo $target_dir

mkdir -p $target_dir
#python normalize_and_filter_text.py --lang $lg < $text_path | grep -v '\-\-\-' >! $target_dir/lm.upper.lid.txt
#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/lm.upper.lid.txt --only-source --destdir $target_dir --thresholdsrc 2 --padding-factor 1 --dict-only
#cut -f1 -d' ' $target_dir/dict.txt | grep -v -x '[[:punct:]]*' | grep -Pv '\d\d\d\d\d+' >! $target_dir/words.txt
cp /kaggle/input/wav2vec-u-cv-swedish-audio/train.wrd $target_dir/lm.upper.lid.txt
cut -f1 -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/words.txt

#one=$(echo "1" | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -p ' ' -w '' -l $ph_lg --language-switch remove-flags)
#sed 's/$/ 1/' $target_dir/words.txt | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -o $target_dir/phones.txt -p ' ' -w '' -l $ph_lg -j 70 --language-switch remove-flags
cut -f2- -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/phones.txt

#echo "one is ${one}"

#sed -i "s/${one}$//" $target_dir/phones.txt
#paste $target_dir/words.txt $target_dir/phones.txt >! $target_dir/lexicon.lst
cp /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train $target_dir/lexicon.lst

#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 1000 --padding-factor 1 --dict-only
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 2 --padding-factor 1 --dict-only

python filter_lexicon.py -d $target_dir/phones/dict.txt < $target_dir/lexicon.lst >! $target_dir/lexicon_filtered.lst
python phonemize_with_sil.py -s 0.25 --surround --lexicon $target_dir/lexicon_filtered.lst < $target_dir/lm.upper.lid.txt >! $target_dir/phones/lm.phones.filtered.txt
cp $target_dir/phones/dict.txt $target_dir/phones/dict.phn.txt
echo "<SIL> 0" >> $target_dir/phones/dict.phn.txt
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones/lm.phones.filtered.txt --workers 70 --only-source --destdir $target_dir/phones --srcdict $target_dir/phones/dict.phn.txt

lmplz -o 4 < $target_dir/lm.upper.lid.txt --discount_fallback --prune 0 0 0 3 >! $target_dir/kenlm.wrd.o40003.arpa
build_binary $target_dir/kenlm.wrd.o40003.arpa $target_dir/kenlm.wrd.o40003.bin
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words_sil lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn'" "kaldi_root='/opt/kaldi'"
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones  "in_labels='phn'" "kaldi_root='/opt/kaldi'"

lmplz -o 4 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.04.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.04.arpa $target_dir/phones/lm.phones.filtered.04.bin
lmplz -o 6 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.06.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.06.arpa $target_dir/phones/lm.phones.filtered.06.bin

lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_phn_sil lm_arpa=$target_dir/phones/lm.phones.filtered.06.arpa data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn'" "kaldi_root='/opt/kaldi'"

Overwriting prepare_text.sh


`add-self-loop-simple.cc` attempts to use `std::endl` with `KALDI_LOG`, which doesn't work, so rewrite that (I'm not sure if this actually prevents anything from working, but it is really distracting).

In [ ]:
%%writefile /tmp/fairseq/examples/speech_recognition/kaldi/add-self-loop-simple.cc
/*
* Copyright (c) Facebook, Inc. and its affiliates.
*
* This source code is licensed under the MIT license found in the
* LICENSE file in the root directory of this source tree.
*/

#include <iostream>
#include "fstext/fstext-lib.h" // @manual
#include "util/common-utils.h" // @manual

/*
 * This program is to modify a FST without self-loop by:
 *   for each incoming arc with non-eps input symbol, add a self-loop arc
 *   with that non-eps symbol as input and eps as output.
 *
 * This is to make sure the resultant FST can do deduplication for repeated
 * symbols, which is very common in acoustic model
 *
 */
namespace {
int32 AddSelfLoopsSimple(fst::StdVectorFst* fst) {
  typedef fst::MutableArcIterator<fst::StdVectorFst> IterType;

  int32 num_states_before = fst->NumStates();
  fst::MakePrecedingInputSymbolsSame(false, fst);
  int32 num_states_after = fst->NumStates();
  KALDI_LOG << "There are " << num_states_before
            << " states in the original FST; "
            << " after MakePrecedingInputSymbolsSame, there are "
            << num_states_after << " states ";

  auto weight_one = fst::StdArc::Weight::One();

  int32 num_arc_added = 0;

  fst::StdArc self_loop_arc;
  self_loop_arc.weight = weight_one;

  int32 num_states = fst->NumStates();
  std::vector<std::set<int32>> incoming_non_eps_label_per_state(num_states);

  for (int32 state = 0; state < num_states; state++) {
    for (IterType aiter(fst, state); !aiter.Done(); aiter.Next()) {
      fst::StdArc arc(aiter.Value());
      if (arc.ilabel != 0) {
        incoming_non_eps_label_per_state[arc.nextstate].insert(arc.ilabel);
      }
    }
  }

  for (int32 state = 0; state < num_states; state++) {
    if (!incoming_non_eps_label_per_state[state].empty()) {
      auto& ilabel_set = incoming_non_eps_label_per_state[state];
      for (auto it = ilabel_set.begin(); it != ilabel_set.end(); it++) {
        self_loop_arc.ilabel = *it;
        self_loop_arc.olabel = 0;
        self_loop_arc.nextstate = state;
        fst->AddArc(state, self_loop_arc);
        num_arc_added++;
      }
    }
  }
  return num_arc_added;
}

void print_usage() {
  std::cout << "add-self-loop-simple usage:\n"
               "\tadd-self-loop-simple <in-fst> <out-fst> \n";
}
} // namespace

int main(int argc, char** argv) {
  if (argc != 3) {
    print_usage();
    exit(1);
  }

  auto input = argv[1];
  auto output = argv[2];

  auto fst = fst::ReadFstKaldi(input);
  auto num_states = fst->NumStates();
  KALDI_LOG << "Loading FST from " << input << " with " << num_states
            << " states.";

  int32 num_arc_added = AddSelfLoopsSimple(fst);
  KALDI_LOG << "Adding " << num_arc_added << " self-loop arcs ";

  fst::WriteFstKaldi(*fst, std::string(output));
  KALDI_LOG << "Writing FST to " << output;

  delete fst;
}

Overwriting /tmp/fairseq/examples/speech_recognition/kaldi/add-self-loop-simple.cc


In [ ]:
!zsh prepare_text.sh sv /kaggle/working/sentences.txt /kaggle/working/preppedtext

sv
sv
/kaggle/working/sentences.txt
/kaggle/working/preppedtext
=== 1/5 Counting and sorting n-grams ===
Reading /kaggle/working/preppedtext/lm.upper.lid.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 14359 types 3160
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:37920 2:2571431424 3:4821433856 4:7714294272
Statistics:
1 3160 D1=0.722623 D2=1.14413 D3+=1.45956
2 10285 D1=0.848104 D2=1.2466 D3+=1.46191
3 12632 D1=0.943362 D2=1.24166 D3+=1.32723
4 19/11699 D1=0.970399 D2=1.4843 D3+=2.12351
Memory estimate for binary LM:
type     kB
probing 617 assuming -p 1.5
probing 764 assuming -r models -p 1.5
trie    309 without quantization
trie    182 assuming -q 8 -b 8 quantization 
trie    293 assuming -a 22 array pointer compression
trie    166 assuming -a 22 -q 8 -b 8 array pointer compression and quant